In [1]:
!pip install openpyxl
# !echo Artur119289 | sudo -S pip3 install --user --upgrade pandas
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import re
import urllib3
import urllib.parse
from urllib.request import urlopen
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import progressbar
import time
from requests.auth import HTTPBasicAuth

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


/tmp/ipykernel_42210/3575917780.py:15: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_status(logs):
    for log in logs:
        if log['message']:
            d = json.loads(log['message'])
            try:
                content_type = 'text/html' in d['message']['params']['response']['headers']['content-type']
                response_received = d['message']['method'] == 'Network.responseReceived'
                if content_type and response_received:
                    return d['message']['params']['response']['status']
            except:
                pass

def preprocess(line):
    return ' '.join(line.replace('\r',' ').replace('\xa0',' ').replace('\xad', ' ').strip().split())

def get_text(soup):
    text = []
    for script in soup(["script", "style","footer","input",'link','meta','button']):
        script.extract()

    paragraph = '\n'.join(list(dict.fromkeys([text for text in soup.stripped_strings])))
    return paragraph


def get_all_links(url,link_hierarchy,soup):
    child_links = []
    for script in soup(["script", "style"]):
        script.extract()
    
    sublinks = [a.get('href') for a in soup.find_all('a') if a.get('href') and a.get('href').startswith('/')]
    
    sublinks = [link[1:] for link in sublinks]
    sublinks = [*set(sublinks)]
    sublinks = list(filter(None, sublinks))
    
    for link in sublinks:
        if ".pdf" not in link or ".PDF" not in link:
            if 'www' in link:
                new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
            else:
                new_url = os.path.join(url[:url.rfind('//')+2], urlparse(url).netloc, link)
            child_links.append(new_url)
    
    link_hierarchy[url] = child_links
    return

def get_responce(links):
    responses = []
    selenium = []
    status = 0
    page = ''
    header = {
  'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
  'accept-encoding':'gzip, deflate, br',
  'accept-language':'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
  'cache-control':'no-cache',
  'dnt': '1',
  'pragma': 'no-cache',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'none',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    for link in links:
        try:
            page = requests.get(link,headers=header,auth=HTTPBasicAuth('user', 'pass'),verify=False)
            status = page.status_code
        except:
            print('Cannot reach website')
        if status == 401:    
            print(link, status, sep=' : ')
        if status == 200:
            page.encoding = 'utf-8'
            responses.append((link, page.text))
            print(link, status, sep=' : ')
        
        else:
            page.encoding = 'utf-8'
            selenium.append(link)
            print(link, status, sep=' : ')
        status = 0

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options = chrome_options)

In [8]:
DataSet = pd.read_csv('PreprocessedESG.csv')

In [10]:
DataSet = DataSet.drop(['Unnamed: 0','Unnamed: 15'],axis=1)

In [11]:
DataSet

,Бренд и Юрлицо (!),Вебсайт,ИНН,Регион,"Устойчивое развитие, Социальная отчетность, ESG",Экология,"Забота о потребителях, защита прав потребителей","Забота о работниках, социальные программы","Поставщикам, Закупки","Корпоративное управление, совет директоров, высший менеджмент","Миссия, ценности",Этический кодекс,Антикоррупция,Раскрытие информации,Количесво ссылок
0,Адмиралтейские верфи,http://admship.ru/,7839395419,СЗФО,NaN,нет,нет,http://admship.ru/job/,http://admship.ru/purchase/,нет,http://admship.ru/about/corruption/,нет,http://admship.ru/about/corruption/,http://admship.ru/about/disclosure/,5.0
1,Алмаз-Антей,http://www.almaz-antey.ru/,7731084175,СЗФО,NaN,нет,нет,http://www.almaz-antey.ru/kadrovaya-i-sotsialn...,http://www.almaz-antey.ru/zakupochnaya-deyatel...,http://www.almaz-antey.ru/istoriya/,http://www.almaz-antey.ru/strategiya/,http://www.almaz-antey.ru/upload/corruption/%D...,http://www.almaz-antey.ru/anti-corruption/,нет,7.0
2,Армалит,https://armalit.ru/,7805148130,СЗФО,NaN,нет,нет,https://armalit.ru/about/training-center/,нет,https://armalit.ru/about/management/,нет,нет,нет,https://armalit.ru/information-disclosure/,3.0
3,Аттика,https://attikarus.ru/,7806521932,СЗФО,NaN,нет,нет,нет,https://attikarus.ru/about/suppliers/,https://attikarus.ru/about/team/,https://attikarus.ru/about/history/,нет,https://attikarus.ru/anti-corruption-policy/,нет,5.0
4,Балтийский завод,https://www.bz.ru/,7830001910,СЗФО,NaN,нет,нет,https://www.bz.ru/career/social-policy/,https://www.bz.ru/partners/purchases/,https://www.bz.ru/about/ceo/,нет,нет,https://www.bz.ru/protivodeystvie-korruptsii/,https://www.bz.ru/partners/disclosure-of-infor...,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,«Эфирное»,https://www.efko.ru,NaN,RAEX,NaN,NaN,NaN,NaN,NaN,NaN,https://www.efko.ru/about/#mission,NaN,NaN,NaN,NaN
306,«Торговая компания «Мираторг»,http://www.miratorg.ru/,NaN,RAEX,https://miratorg.ru/about/sustainable_developm...,NaN,NaN,NaN,NaN,https://miratorg.ru/about/directors/,NaN,NaN,NaN,NaN,NaN
307,РОСВОДОКАНАЛ,http://www.rosvodokanal.ru/,NaN,RAEX,https://www.rosvodokanal.ru/ustoychivoe-razvit...,NaN,NaN,NaN,NaN,https://www.rosvodokanal.ru/about/rukovodstvo/,https://www.rosvodokanal.ru/about/missiya/,NaN,NaN,NaN,NaN
308,«Мосводоканал»,http://www.mosvodokanal.ru/,NaN,RAEX,https://www.mosvodokanal.ru/about/evolution/ec...,NaN,NaN,NaN,NaN,https://www.mosvodokanal.ru/about/leadership/,NaN,NaN,https://www.mosvodokanal.ru/about/anti-corrupt...,NaN,NaN


In [12]:
get_responce(list(DataSet[' Вебсайт'].values))

KeyError: 'Вебсайт'

In [ ]:
get_responce(list(DataSet[' Вебсайт']))

http://admship.ru/ : 200
http://www.almaz-antey.ru/ : 200


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'armalit.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Cannot reach website
https://armalit.ru/ : 0


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'attikarus.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://attikarus.ru/  : 404
https://www.bz.ru/ : 200


...

https://nordsy.spb.ru/ : 200


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'power-m.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://power-m.ru/ : 200
https://onegoshipyard.ru/ : 200


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'onegoshipyard.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kirovets-ptz.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://kirovets-ptz.com/ : 200
